In [ ]:
#IMPORTANT NOTE
#HERE WE EXECUTE THE QUERY USING sf_cur.execute(query, (JOB_RUN_ID, JOB_ID,JOB_RUN_ID))
#NOTICE THAT JOB_RUN_ID IS USED TWICE BECAUSE THE SQL EXPECTS THE JOB_RUN_ID VALUE AT TWO PLACES

import os
import sys
import snowflake.connector as sf_c
import pandas as pd
from datetime import datetime
import concurrent.futures
import boto3
import fnmatch
import zipfile
from io import BytesIO
import tarfile
import subprocess

s3_resource = boto3.resource('s3')
s3_client = boto3.client('s3')

CEDL_HOME=os.environ['CEDL_HOME']
nexus_connectionProfile = CEDL_HOME + '/etc/.sf.nexus.profile'
s3_connectionProfile = CEDL_HOME + '/etc/.s3_connection_profile'


def snowFlake_Connection():
    try:
        pathExist=os.path.exists(nexus_connectionProfile)
        if(not pathExist):
            print('The profile {} doesn''t exist'.format(nexus_connectionProfile))
            exit(1)
        profileFile=open(nexus_connectionProfile)
        for line in profileFile:
            if(line.split('=')[0]=='snowflakeAccount'):
                snowflakeAccount=line.split('=')[1].replace('\n','')
            elif (line.split('=')[0]=='snowflakeUsername'):
                snowflakeUsername=line.split('=')[1].replace('\n','')
            elif (line.split('=')[0]=='snowflakePassword'):
                snowflakePassword=line.split('=')[1].replace('\n','')
            elif (line.split('=')[0]=='snowflakeRole'):
                snowflakeRole=line.split('=')[1].replace('\n','')
            elif (line.split('=')[0]=='snowflakeDBName'):
                snowflakeDBName=line.split('=')[1].replace('\n','')
            elif (line.split('=')[0]=='snowflakeWarehouse'):
                snowflakeWarehouse=line.split('=')[1].replace('\n','')
            else:
                pass
        profileFile.close()
        if(len(snowflakeAccount)==0 or len(snowflakeUsername)==0 or len(snowflakePassword)==0 or len(snowflakeRole)==0 or len(snowflakeDBName)==0 or len(snowflakeWarehouse)==0):
            print('some parameters are missing from {}'.format(nexus_connectionProfile))
            exit(1)
        conn=sf_c.connect(user=snowflakeUsername, password=snowflakePassword, account=snowflakeAccount, warehouse=snowflakeWarehouse, database=snowflakeDBName)
        print("connected to SNOWFLAKE Database.")
    except sf_c.Error as e:
        print('Error connecting to SNOWFLAKE Database - {}'.format(e))
        exit(1)
    return conn



def get_job_details(conn, JOB_RUN_ID,JOB_ID):
    try:
        sf_cur = sf_conn.cursor()
        query = """
            SELECT NVL(B.ROWS_PROCESSED, 0),
                NVL(A.RUN_STATUS, 'Failed')
            FROM (
                SELECT JOB_RUN_ID,
                    CASE
                        WHEN SUM(CASE
                                    WHEN STATUS = 'S'
                                        THEN 0
                                    ELSE 1
                                    END) = 0
                            THEN 'Succeeded'
                        ELSE 'Failed'
                        END AS RUN_STATUS
                FROM NEXUS.JMAN.vw_TABLE_REFRESH_WHITE_LIST
                WHERE IS_ACTIVE = 'Y'
                    AND JOB_RUN_ID = %s
                    AND JOB_ID = %s
                GROUP BY JOB_RUN_ID
                ) A
            LEFT JOIN (
                SELECT JOB_RUN_ID,
                    SUM(TARGET_SUCCESS_ROWS) AS ROWS_PROCESSED
                FROM JMAN.vw_FEEDCOUNT
                WHERE JOB_RUN_ID = %s
                    AND STATUS = 'S'
                GROUP BY JOB_RUN_ID
                ) B ON A.JOB_RUN_ID = B.JOB_RUN_ID
            """
        #IMPORTANT NOTE WE USE    JOB_RUN_ID TWICE
        sf_cur.execute(query, (JOB_RUN_ID, JOB_ID,JOB_RUN_ID))
        result = sf_cur.fetchone()

        if sf_cur.rowcount == 0:
            print('No Data available')
            exit(1)
        elif sf_cur.rowcount > 1:
            print('More than one row returned')
            exit(1)
        else:
            job_run_id,  status  = result
    except Exception as e:
        print('Error getting job details: {}'.format(e))
        exit(1)
    return  job_run_id, status



sf_conn = snowFlake_Connection()

JOB_RUN_ID = 507249
JOB_ID = 'LOAD_CLAIMS_TMG_MCARE'


result = get_job_details(sf_conn,JOB_RUN_ID,JOB_ID)
print(f"Values returned from the function - > {result}")
